<a href="https://colab.research.google.com/github/dwelsh22/Data_Mining_Water_Table_Driven_Data/blob/master/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import researchpy as rp
import scipy.stats as stats

# Import and clean data
training_set = pd.read_csv("training_set_values.csv")
training_labels = pd.read_csv("training_set_labels.csv")

# Clean Data
training_set['date_recorded'] = [pd.Timestamp(date) for date in training_set['date_recorded']]
training_set.set_index('id', drop=True, inplace=True)
training_labels.set_index('id', drop=True, inplace=True)
training_set['status_group'] = training_labels['status_group']

print(rp.summary_cont(training_set['amount_tsh'].groupby(training_set['status_group'])))
print(stats.f_oneway(training_set['amount_tsh'][training_set['status_group'] == 'functional'],
               training_set['amount_tsh'][training_set['status_group'] == 'functional needs repair'],
               training_set['amount_tsh'][training_set['status_group'] == 'non functional']))

def generateCrosstab_chi_squared(test_against, test_series):
    results = list()
    for label, content in test_series.iteritems():
        if content.dtype == 'object': #check datatype before performing check
            crosstab, test_results, expected = rp.crosstab(test_against, content, test= "chi-square", expected_freqs= True, prop= "cell")
            print(label + ": " + str(test_results['results'][2])) #create a dataframe containing all Cramer's V
            results.append((label, crosstab, test_results, expected))
        else:
            continue

    return results

def draw_box_plot():
    # df_box = df.copy()
    # training_set['date'] = df_box.index
    # df_box.reset_index(inplace=True)
    # df_box['year'] = [d.year for d in df_box.date]
    # df_box['month'] = [d.strftime('%b') for d in df_box.date]
    # df_box.month = pd.Categorical(df_box.month, categories=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], ordered=True)

    fig, ax = plt.subplots(figsize=(16,6))
    plt.ylim(1960, 2020)

    ax = sns.boxplot(x='status_group', y='construction_year', data=training_set, ax=ax)
    ax.set_xlabel('Functionality')
    ax.set_ylabel('TSH')
    ax.set_title('correlation graph')

    # Save image and return fig (don't change this part)
    fig.savefig('box_plot.png')
    return fig

draw_box_plot()

# results = generateCrosstab_chi_squared(training_set['status_group'], training_set)

training_set.drop(columns=['permit']) #0.0347
training_set.drop(columns=['management_group']) #0.0492, also group of management, not needed
training_set.drop(columns=['recorded_by']) #All one entry, completely useless
training_set.drop(columns=['quality_group'])
training_set.drop(columns=['quantity_group'])
training_set.drop(columns=['payment_type'])
training_set.drop(columns=['waterpoint_type_group'])

# Correlation map
# corr = training_set.corr(method='kendall')
# print(corr)

training_set = training_set.groupby(['status_group', 'amount_tsh', 'quantity_group', 'construction_year'], as_index=False).count()

test_set = pd.read_csv("test_set_values.csv")
submission = pd.read_csv("SubmissionFormat.csv")

# find appropriate graph to determine what kind of regression and prediction needs to happen
